# Projeto 2 - Ciência dos Dados

Nome: Carlos Andrade Inacio

Nome: Carlos Felipe Borges Mesquita

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Carlos\Documents\insper\cdados\Projeto 2\cdados_projeto2


Carregando a base de dados

In [3]:
filename = 'AllResults1.csv'

In [4]:
train = pd.read_csv(filename)
train

,PPID,PP,EAL,SEN,HML,Re,Wr,Ma,Att8Est,Att8Act,Att8Diff,EngEst,EngAct,EngDiff,MathsEst,MathsAct,MathsDiff,EbaccEst,EbaccAct,EbaccDiff
0,617,1.0,0,0.0,NaN,NaN,NaN,NaN,NaN,1.9,NaN,NaN,1.5,NaN,NaN,2.0,NaN,NaN,0.0,NaN
1,283,1.0,1,0.0,NaN,NaN,NaN,NaN,NaN,4.3,NaN,NaN,3.0,NaN,NaN,5.5,NaN,NaN,3.7,NaN
2,75,0.0,0,0.0,M,NaN,NaN,3,2.9,4.1,1.17,3.5,4.0,0.54,2.9,4.0,1.13,1.9,1.3,-0.57
3,1019,1.0,0,0.0,L,NaN,NaN,3,2.2,3.5,1.26,2.8,4.0,1.18,2.0,3.0,0.98,1.2,2.0,0.84
4,474,0.0,0,0.0,L,3C,3B,N,1.7,2.9,1.21,2.2,3.0,0.76,1.3,3.0,1.65,0.7,2.0,1.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032,174,1.0,0,0.0,M,4B,4,4B,3.9,5.0,1.08,4.3,6.0,1.67,3.6,4.0,0.36,3.5,4.3,0.82
1033,812,1.0,0,0.0,M,5B,3,4C,4.3,3.1,-1.18,4.7,3.0,-1.71,4.1,3.0,-1.08,4.0,3.0,-0.97
1034,425,1.0,0,1.0,H,5B,4,5C,5.5,5.2,-0.30,5.8,6.0,0.20,5.4,4.0,-1.44,5.4,5.3,-0.05
1035,376,1.0,0,0.0,M,5C,4,4B,4.7,1.7,-3.03,5.1,4.0,-1.13,4.6,3.0,-1.58,4.5,1.0,-3.49


In [5]:
train = train.drop(columns=['PPID','Att8Est','Att8Act','Att8Diff','EngEst','EngAct','EngDiff','MathsEst','MathsAct','MathsDiff','EbaccEst','EbaccAct','EbaccDiff'])
train

,PP,EAL,SEN,HML,Re,Wr,Ma
0,1.0,0,0.0,NaN,NaN,NaN,NaN
1,1.0,1,0.0,NaN,NaN,NaN,NaN
2,0.0,0,0.0,M,NaN,NaN,3
3,1.0,0,0.0,L,NaN,NaN,3
4,0.0,0,0.0,L,3C,3B,N
...,...,...,...,...,...,...,...
1032,1.0,0,0.0,M,4B,4,4B
1033,1.0,0,0.0,M,5B,3,4C
1034,1.0,0,1.0,H,5B,4,5C
1035,1.0,0,0.0,M,5C,4,4B


In [6]:
# PP = Pupil Premium - The pupil premium is additional funding for publicly funded schools in England.
# EAL = English as an additional language
# SEN = Special Education Needs (True or False)
# HML = Ability based on previous Data (Higher, Middle, Lower)
# Re = Reading Grade from age 10
# Wr = Writing Grade from age 10
# Ma = Maths Grade from age 10

In [31]:
# numerificação das notas 
# https://www.northoxfordshire-academy.org/Portals/0/Letters%20Sent%20Home/Target%20Setting%20Year%2011%20Explained%202016%2017.pdf?ver=2016-10-31-144606-610
grades = {'N' : '0', 'B' : '5', '3C' : '4', '3B' : '4', '3A' : '4', '4C' : '5', '4B' : '5', '4A' : '6', '5C' : '7', '5B' : '8', '5A' : '8', '6C' : '8', '6B' : '9', '6A' : '10'}

df = train

def update_grades(column, dataframe):
    for k, v in enumerate(dataframe[column]):
        if v in grades.keys():
            dataframe[column][k] = grades[v]

def update_average(dataframe, index):
    count = 0
    average = 0
    Re = -1 if pd.isna(dataframe['Re'][index]) else int(dataframe['Re'][index])
    Wr = -1 if pd.isna(dataframe['Wr'][index]) else int(dataframe['Wr'][index])
    Ma = -1 if pd.isna(dataframe['Ma'][index]) else int(dataframe['Ma'][index])

    if Re >= 0:
        count += 1
        average += Re
    if Wr >= 0:
        count += 1
        average += Wr
    if Ma >= 0:
        count += 1
        average += Ma

    if count > 0:
        average /= count
    
    return average
            
update_grades('Re', df)
update_grades('Wr', df)
update_grades('Ma', df)       


# criar coluna de média
average = []
for i in range(len(df['Re'])):
    average.append(update_average(df, i))

df['AVG'] = average

,PP,EAL,SEN,HML,Re,Wr,Ma,AVG
0,1.0,0,0.0,NaN,NaN,NaN,NaN,0.000000
1,1.0,1,0.0,NaN,NaN,NaN,NaN,0.000000
2,0.0,0,0.0,M,NaN,NaN,3,3.000000
3,1.0,0,0.0,L,NaN,NaN,3,3.000000
4,0.0,0,0.0,L,4,4,0,2.666667
...,...,...,...,...,...,...,...,...
1032,1.0,0,0.0,M,5,4,5,4.666667
1033,1.0,0,0.0,M,8,3,5,5.333333
1034,1.0,0,1.0,H,8,4,7,6.333333
1035,1.0,0,0.0,M,7,4,5,5.333333


In [36]:
# correlação média com PP
pp_avg = df.loc[:,["PP", "AVG"]]
corr = pp_avg.corr()

print('A correlação entre o PP e a média das notas é negativa, o que indica que se o aluno pagar o financiamento adicional, a média de notas dele é menor.')
corr

A correlação entre o PP e a média das notas é negativa, o que indica que se o aluno pagar o financiamento adicional, a média de notas dele é menor.


,PP,AVG
PP,1.000000,-0.119484
AVG,-0.119484,1.000000


In [37]:
# correlação média com EAL
eal_avg = df.loc[:,["EAL", "AVG"]]
corr = eal_avg.corr()

print('A correlação entre o EAL e a média das notas é negativa, o que indica que se o inglês não for a linguagem nativa do estudante, suas notas são mais baixas.')
corr

A correlação entre o EAL e a média das notas é negativa, o que indica que se o inglês não for a linguagem nativa do estudante, suas notas são mais baixas.


,EAL,AVG
EAL,1.000000,-0.206433
AVG,-0.206433,1.000000


In [39]:
# correlação média com SEN
sen_avg = df.loc[:,["SEN", "AVG"]]
corr = sen_avg.corr()

print('A correlação entre o SEN e a média das notas é negativa, o que indica que se o aluno tiver necessidades especiais, suas notas não menores.')
corr

A correlação entre o SEN e a média das notas é negativa, o que indica que se o aluno tiver necessidades especiais, suas notas não menores.


,SEN,AVG
SEN,1.000000,-0.114002
AVG,-0.114002,1.000000
